# Speed Skydiving scoring bulk files processor
Uses the `./data` directory as a data lake.  It finds all valid FlySight files in that directory, from 0 to as many as it finds and fit in memory, and process them all.  Summary of "all files" or "all jumps" at the bottom of the document.

In [1]:
from collections import namedtuple
from copy import deepcopy

from ssscoring.constants import BREAKOFF_ALTITUDE
from ssscoring.constants import FT_IN_M
from ssscoring.constants import PERFORMANCE_WINDOW_LENGTH
from ssscoring.fs1 import aggregateResults
from ssscoring.fs1 import convertFlySight2SSScoring
from ssscoring.fs1 import dropNonSkydiveDataFrom
from ssscoring.fs1 import getAllSpeedJumpFilesFrom
from ssscoring.fs1 import getSpeedSkydiveFrom
from ssscoring.fs1 import isValidJump
from ssscoring.fs1 import isValidMinimumAltitude
from ssscoring.fs1 import jumpAnalysisTable
from ssscoring.fs1 import processAllJumpFiles
from ssscoring.fs1 import roundedAggregateResults
from ssscoring.fs1 import totalResultsFrom
from ssscoring.fs1 import validFlySightHeaderIn
from ssscoring.notebook import SPEED_COLORS
from ssscoring.notebook import graphAltitude
from ssscoring.notebook import graphAngle
from ssscoring.notebook import graphJumpResult
from ssscoring.notebook import initializeExtraYRanges
from ssscoring.notebook import initializePlot

import csv
import os
import os.path as path

import bokeh.plotting as bp
import bokeh.models as bm
import ipywidgets as widgets
import pandas as pd

In [2]:
DATA_LAKE_ROOT = './data'

---
## <span style="color: yellow">IMPORTANT</span> - Set the drop zone altitude MSL

Set the value in ft (even though SSScoring uses meters) out of convenience because all DZs in the US, many in Asia and Mexico, and many in Europe express the altitude in feet.

### Some DZs

| Drop zone | Alt (ft) |
|-----------|----------|
| Bay Area Skydiving | 23 |
| Skydive Arizona | 1509 |
| Thai Sky Adventures | 21 |

In [3]:
dropZoneAltMSL = 23

In [4]:
dropZoneAltMSLMeters = dropZoneAltMSL/FT_IN_M
display(widgets.HTML('<h2>DZ Altitude = <span style = "color: green">%7.2f ft</span> (%7.2f m)<h1>' % (dropZoneAltMSL, dropZoneAltMSLMeters)))

HTML(value='<h2>DZ Altitude = <span style = "color: green">  23.00 ft</span> (   7.01 m)<h1>')

In [5]:
jumpFiles = getAllSpeedJumpFilesFrom(DATA_LAKE_ROOT)

In [6]:
jumpResults = processAllJumpFiles(jumpFiles, altitudeDZMeters = dropZoneAltMSLMeters)

---
## Results

In [7]:
aggregate = aggregateResults(jumpResults)
sumResults = totalResultsFrom(aggregate)
display(aggregate)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
00-00-00-EC-baseline,472.446,177.048,295.560,397.872,457.092,475.164,25.0,476.712
11-07-54,464.094,160.632,302.076,408.600,457.164,464.112,25.0,465.804
12-08-44,457.524,172.332,290.232,386.100,439.560,459.792,25.0,460.152


### Rounded results for training log

In [8]:
roundedResults = roundedAggregateResults(jumpResults)

### All jumps

In [9]:
def dumpSimple(table: pd.DataFrame):
    fileName = os.path.join(DATA_LAKE_ROOT, resultRef+'.tsv')
    simpleTable = table.copy()
    simpleTable = simpleTable.drop('speedAngle', axis = 1)
    simpleTable.to_csv(fileName, sep = '\t', index = False)

In [10]:
def displayJumpDataIn(resultsTable: pd.DataFrame):
    table = resultsTable.copy()
    # Experimental
    # For more information on the `interpolate` method and its options, see the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html).
    # Additionally, you can also consider using other libraries like `scipy` which provides more advanced interpolation methods, such as `interp1d` or `griddata`. See the [scipy documentation](https://docs.scipy.org/doc/scipy/reference/interpolate.html) for more information.
    table.vKMh = table.vKMh.apply(round)
    table.hKMh = table.hKMh.apply(round)
    table['altitude (ft)'] = table['altitude (ft)'].apply(round)
    table.netVectorKMh = table.netVectorKMh.apply(round)
    table.index = ['']*len(table)
    display(table)
    # dumpSimple(table)

In [11]:
allJumpsPlot = initializePlot('All jumps in set')
jumpNumber = 0
mixColor = 0
for resultRef in sorted(jumpResults.keys()):
    jumpResult = jumpResults[resultRef]
    if jumpResult.score > 0.0:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - score = %.02f km/h</span></h1>' % (jumpResult.color, resultRef, jumpResult.result, jumpResult.score)        
    else:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s</span></h1>' % (jumpResult.color, resultRef, jumpResult.result)
    display(widgets.HTML(validJumpStatus))

    maxSpeed = jumpResult.maxSpeed
    window = jumpResult.window
    mixColor = (mixColor+1)%len(SPEED_COLORS)
    if jumpResult.score > 0.0:
        display(widgets.HTML('<h3>Max speed = {0:,.0f}; '.format(maxSpeed)+('exit at %d m (%d ft), end scoring window at %d m (%d ft)'%(window.start, 3.2808*window.start, window.end, 3.2808*window.end))))
        if not isValidMinimumAltitude(jumpResult.data.altitudeASL.max()):
            display(widgets.HTML('<span style="color: yellow"><span style="font-weight: bold">Warning:</span> exit altitude ASL was lower than the minimum scoring altitude according to IPC and USPA.'))
        display(widgets.HTML('</h3>'))
        # TODO: Move this to an in-notebook function or module.
        displayJumpDataIn(jumpResult.table)
        individualPlot = initializePlot(resultRef)
        individualPlot = initializeExtraYRanges(individualPlot, startY = min(jumpResult.data.altitudeASLFt)-500.0, endY = max(jumpResult.data.altitudeASLFt)+500.0)
        graphAltitude(individualPlot, jumpResult)
        graphAngle(individualPlot, jumpResult)
        hoverValue = bm.HoverTool(tooltips = [ ('Y-val', '@y{0.00}'), ])
        individualPlot.add_tools(hoverValue)
        graphJumpResult(individualPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[0])
        graphJumpResult(allJumpsPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[mixColor],
                        legend = '%s - %.2f' % (resultRef, jumpResult.score),
                        showIt = False)

HTML(value='<hr><h1><span style="color: #0f0">00-00-00-EC-baseline jump - 🟢 valid - score = 472.45 km/h</span>…

HTML(value='<h3>Max speed = 477; exit at 4237 m (13902 ft), end scoring window at 1981 m (6500 ft)')

HTML(value='</h3>')

,time,vKMh,hKMh,speedAngle,netVectorKMh,altitude (ft)
,5.0,177,78,66.2,193,13475
,10.0,296,68,77.0,303,12383
,15.0,398,78,78.8,406,10791
,20.0,457,49,83.9,460,8823
,25.0,475,32,86.1,476,6686


HTML(value='<hr><h1><span style="color: #0f0">11-07-54 jump - 🟢 valid - score = 464.09 km/h</span></h1>')

...

,time,vKMh,hKMh,speedAngle,netVectorKMh,altitude (ft)
,5.0,161,113,55.0,196,12786
,10.0,302,69,77.2,310,11652
,15.0,409,48,83.3,411,9926
,20.0,457,35,85.6,459,7913
,25.0,464,25,86.9,465,5848


HTML(value='<hr><h1><span style="color: #0f0">12-08-44 jump - 🟢 valid - score = 457.52 km/h</span></h1>')

HTML(value='<h3>Max speed = 460; exit at 4029 m (13220 ft), end scoring window at 1773 m (5818 ft)')

HTML(value='</h3>')

,time,vKMh,hKMh,speedAngle,netVectorKMh,altitude (ft)
,5.0,172,94,61.3,197,12806
,10.0,290,99,71.2,307,11717
,15.0,386,99,75.6,399,10155
,20.0,440,95,77.8,450,8209
,25.0,460,56,83.1,463,6178


---
## All speed skydives in data lake summary

In [12]:
display(roundedResults)
display(sumResults)
bp.show(allJumpsPlot)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
00-00-00-EC-baseline,472,177,296,398,457,475,25.0,477
11-07-54,464,161,302,409,457,464,25.0,466
12-08-44,458,172,290,386,440,460,25.0,460


,totalSpeed,meanSpeed,maxScore
totalSpeed,1394.064,464.688,472.446
